## 1. Data Preprocessing 

### 1.1 Load Libraries

In [47]:
import pandas as pd
import numpy as np
import pprint
from transformers import DistilBertTokenizer, DistilBertModel

from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
import hnswlib

# min rating to consider
min_rating = 8
# top k popular books
top_k_popular_books = 10_000

# parameters
output_dimension = 64
batch_size = 256
learning_rate = 0.1


### 1.2 Load Dataset

In [48]:
# Datasets
books = pd.read_csv("dataset/Books.csv")

ratings = pd.read_csv("dataset/Ratings.csv")

users = pd.read_csv("dataset/Users.csv")

/var/folders/66/b29gfxwj3tb43t74fmwm28nmrsc95x/T/ipykernel_18184/2153888385.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("dataset/Books.csv")


In [49]:
# Visualization and Type Standardization
users["User-ID"] = users["User-ID"].apply(lambda x: f"user_{x}")

# Filter out books with missing or corrupted information
books["ISBN"] = books["ISBN"].apply(lambda x: f"book_{x}")
books.drop(["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1, inplace=True)
books.dropna(inplace=True)

def clean_year(year):
    try:
        return int(year)
    except ValueError:
        return -1

books['Year-Of-Publication'] = books['Year-Of-Publication'].apply(clean_year)
books = books[books['Year-Of-Publication'] != -1].reset_index(drop=True)
books = books.drop_duplicates(subset='Book-Title')


ratings["ISBN"] = ratings["ISBN"].apply(lambda x: f"book_{x}")
ratings["User-ID"] = ratings["User-ID"].apply(lambda x: f"user_{x}")
ratings["Book-Rating"] = ratings["Book-Rating"].apply(lambda x: float(x))
ratings = ratings[ratings.ISBN.isin(books['ISBN'].unique())]
# Filtering products for simplicity

# Only consider high ratings
ratings = ratings[ratings["Book-Rating"] >= 7]
# Remove outlier users
outlier_threshold = ratings['User-ID'].value_counts().quantile(0.9)
user_rating_count_dict = ratings['User-ID'].value_counts().to_dict()
ratings['rate_count'] = ratings['User-ID'].map(user_rating_count_dict)
ratings = ratings[ratings['rate_count'] <= outlier_threshold]

# Consider user & book that has rating in ratings dataset
books = books[books.ISBN.isin(ratings['ISBN'].unique())]
users = users[users['User-ID'].isin(ratings['User-ID'].unique())]
print(f"Number of Users: {users['User-ID'].nunique()}")
print(f"Number of Books: {books['ISBN'].nunique()}")
print(f"Number of Ratings: {ratings.shape[0]}")


Number of Users: 46810
Number of Books: 41743
Number of Ratings: 85918


In [50]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
1,book_0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,book_0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,book_0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
5,book_0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group
14,book_1558746218,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,1998,Health Communications


### 1.3 Book to Book Matches

In [51]:
# Group books which are read from same user
book_groups_raw = ratings.groupby('User-ID')
book_groups = pd.DataFrame(
    data={
        "User-ID": list(book_groups_raw.groups.keys()),
        "ISBN_list": list(book_groups_raw.ISBN.apply(list)),
    }
)
# Eliminate if user has read one book
book_groups = book_groups[book_groups['ISBN_list'].apply(len) > 1].reset_index(drop=True)
print(f"Number of Groups: {book_groups.shape[0]}")
book_groups.head()


Number of Groups: 16055


,User-ID,ISBN_list
0,user_100004,"[book_0345339703, book_0399146652, book_043906..."
1,user_10003,"[book_068483068X, book_0743446593]"
2,user_100035,"[book_0440211727, book_0671759310]"
3,user_100053,"[book_0312422156, book_0316769487, book_038549..."
4,user_100066,"[book_0060953713, book_0385722206, book_039309..."


In [52]:

book_matches = []
# for each book in our isbn_list we generate pairs
for isbn_list in book_groups['ISBN_list'].values:
    if len(isbn_list) <= 1:
        continue
    for i, main_isbn in enumerate(isbn_list[:-1]):
        for similar_isbn in isbn_list[i+1:]:
            book_matches.append([main_isbn, similar_isbn])

# Dataset generation and visualization
book_pairs_dataset = pd.DataFrame(book_matches, columns=["main_ISBN", "similar_ISBN"])
data_size = book_pairs_dataset.shape[0]
print(f"Number of Matches: {data_size}")
book_pairs_dataset.head()

Number of Matches: 90940


,main_ISBN,similar_ISBN
0,book_0345339703,book_0399146652
1,book_0345339703,book_0439064872
2,book_0345339703,book_059035342X
3,book_0399146652,book_0439064872
4,book_0399146652,book_059035342X


In [53]:
# Our final dataset to train our model 
# Main book features
main_books = books.rename(columns=lambda x: 'main_' + x if x != 'ISBN' else x).copy()
book_pairs = pd.merge(book_pairs_dataset, main_books,
                              left_on='main_ISBN',
                              right_on='ISBN')
book_pairs.drop("ISBN", axis=1, inplace=True)

# Similar book features
similar_books = books.rename(columns=lambda x: 'similar_' + x if x != 'ISBN' else x).copy()
book_pairs = pd.merge(book_pairs, similar_books,
                              left_on='similar_ISBN',
                              right_on='ISBN')

book_pairs.drop("ISBN", axis=1, inplace=True)
book_pairs.head()

,main_ISBN,similar_ISBN,main_Book-Title,main_Book-Author,main_Year-Of-Publication,main_Publisher,similar_Book-Title,similar_Book-Author,similar_Year-Of-Publication,similar_Publisher
0,book_0345339703,book_0399146652,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
1,book_0316284955,book_0399146652,White Oleander : A Novel (Oprah's Book Club),Janet Fitch,2000,Back Bay Books,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
2,book_0312278586,book_0399146652,The Nanny Diaries: A Novel,Emma McLaughlin,2002,St. Martin's Press,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
3,book_0316666343,book_0399146652,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown",The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group
4,book_0140293248,book_0399146652,The Girls' Guide to Hunting and Fishing,Melissa Bank,2000,Penguin Books,The Cat Who Smelled a Rat,Lilian Jackson Braun,2001,Putnam Publishing Group


### 1.4 Convert Dataset to TFDS

In [54]:
batch_size = 256
# Pairs dataset
book_pairs_final = tf.data.Dataset.from_tensor_slices({
    # main book features
    'main_ISBN': tf.cast(book_pairs['main_ISBN'], dtype=tf.string),
    'main_Book-Title': tf.cast(book_pairs['main_Book-Title'], dtype=tf.string),
    'main_Book-Author': tf.cast(book_pairs['main_Book-Author'], dtype=tf.string),
    'main_Publisher': tf.cast(book_pairs['main_Publisher'], dtype=tf.string),

    # similar book features
    'similar_ISBN': tf.cast(book_pairs['similar_ISBN'], dtype=tf.string),
    'similar_Book-Title': tf.cast(book_pairs['similar_Book-Title'], dtype=tf.string),
    'similar_Book-Author': tf.cast(book_pairs['similar_Book-Author'], dtype=tf.string),
    'similar_Publisher': tf.cast(book_pairs['similar_Publisher'], dtype=tf.string),
})

book_pairs_final = book_pairs_final.batch(batch_size)
# Book information dataset
book_infos = tf.data.Dataset.from_tensor_slices({
    'ISBN': tf.cast(books['ISBN'], dtype=tf.string),
    'Book-Title': tf.cast(books['Book-Title'], dtype=tf.string),
    'Book-Author': tf.cast(books['Book-Author'], dtype=tf.string),
    'Year-Of-Publication': tf.cast(books['Year-Of-Publication'], dtype=tf.int32),
    'Publisher': tf.cast(books['Publisher'], dtype=tf.string),
})
book_infos = book_infos.batch(batch_size)


In [55]:
tf.random.set_seed(1002)
train_percentage = 0.8
batch_count = (data_size + batch_size - 1) // batch_size
shuffled = book_pairs_final.shuffle(batch_count, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(batch_count * train_percentage))
test = shuffled.skip(int(batch_count * train_percentage)).take(int(batch_count * (1 - train_percentage)))

## 2. Model Definition

### Lookup Embeddings

In [56]:
print(f"Number of unique books {books['ISBN'].nunique()}")
print(f"Number of unique titles {books['Book-Title'].nunique()}")
print(f"Number of unique authors {books['Book-Author'].nunique()}")
print(f"Number of unique publishers {books['Publisher'].nunique()}")


Number of unique books 41743
Number of unique titles 41743
Number of unique authors 21935
Number of unique publishers 5234


In [57]:
def batch_distilbert_embeddings(book_titles):
    # convert bytes to string
    book_titles = [title.decode('utf-8') for title in book_titles.numpy()]
    # set inputs
    input_ids = tokenizer(book_titles, padding=True, truncation=True, return_tensors='pt', max_length=20)
    # generate embeddings
    outputs = bert_model(input_ids['input_ids'])
    # get last layer
    last_hidden_state = outputs.last_hidden_state

    # mean vector is required and also padded values should be excluded
    input_mask = tf.cast(input_ids['attention_mask'], tf.float32)
    input_mask_expanded = tf.expand_dims(input_mask, -1)
    sum_embeddings = tf.reduce_sum(last_hidden_state.detach().numpy() * input_mask_expanded, axis=1)
    sum_mask = tf.reduce_sum(input_mask_expanded, axis=1)
    mean_embeddings = sum_embeddings / sum_mask
    return book_titles, mean_embeddings

bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

title_list = []
title_embeddings = []

# Process batches of book titles to get embeddings
for batch in book_infos:
    book_titles, embeddings = batch_distilbert_embeddings(batch['Book-Title'])
    title_list.extend(book_titles)
    title_embeddings.extend(embeddings)

title_embeddings_df = pd.DataFrame({'Book-Title':title_list, 'embedding':title_embeddings})


In [58]:
# Book ID Model
book_embedding_dimension = 32
unique_book_ids = books['ISBN'].unique()
book_id_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_ids) + 1, book_embedding_dimension)
])


# Author Model
author_embedding_dimension = 16
unique_book_authors = books['Book-Author'].unique()
book_author_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_authors, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_authors) + 1, author_embedding_dimension)
])

# Publisher Model
publisher_embedding_dimension = 8
unique_book_publishers = books['Publisher'].unique()
book_publisher_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_book_publishers, mask_token=None),
  tf.keras.layers.Embedding(len(unique_book_publishers) + 1, publisher_embedding_dimension)
])


In [59]:
# Book Title Model
title_output_dim = len(title_embeddings[0])
# An embedding if there are any unknown text input occurs
unknown_embedding = np.random.uniform(-1, 1, size=title_output_dim)
title_embeddings.insert(0, unknown_embedding)
book_title_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=title_list, mask_token=None),
  tf.keras.layers.Embedding(
                    input_dim=len(title_list) + 1,
                    output_dim=title_output_dim,
                    embeddings_initializer=tf.keras.initializers.Constant(np.vstack(title_embeddings)),
                    trainable=False)])

In [60]:
# Author Embedding Visualization
book_author_model('Richard Bruce Wright')

<tf.Tensor: shape=(16,), dtype=float32, numpy=
array([-0.00374892,  0.03112172, -0.01137679,  0.01890567,  0.00999472,
        0.03013868, -0.00890771,  0.00090703, -0.03488296,  0.02796206,
        0.0466506 ,  0.0105405 ,  0.04664585,  0.03104696, -0.04421418,
       -0.02597226], dtype=float32)>

### Book Model

In [61]:
class BookModel(tfrs.Model):

  def __init__(self, book_id_model, book_title_model, book_author_model, book_publisher_model):
    super().__init__()
    # assigning sub models to convert ids to embeddings
    self.book_id_model = book_id_model
    self.book_title_model = book_title_model
    self.book_author_model = book_author_model
    self.book_publisher_model = book_publisher_model

  def call(self, features: Dict[Text, tf.Tensor]):

    # concatenation of embeddings
    return tf.concat([
        self.book_id_model(features["ISBN"]),
        self.book_title_model(features['Book-Title']),
        self.book_author_model(features["Book-Author"]),
        self.book_publisher_model(features["Publisher"]),
    ], axis=1)

In [62]:
# Initialization of our book model
book_model = BookModel(book_id_model, book_title_model, book_author_model, book_publisher_model)

# Sample example
book_model({'Book-Author': ['Richard Bruce Wright'],
            'Book-Title': ['Clara Callan'],
            'ISBN': ['book_0002005018'],
            'Publisher': ['HarperFlamingo Canada'],
            'Year-Of-Publication': [2001]})

<tf.Tensor: shape=(1, 824), dtype=float32, numpy=
array([[ 3.51698734e-02, -2.78519467e-03,  2.30444595e-03,
         3.12348045e-02,  1.89812295e-02, -3.99783365e-02,
         1.17850304e-02,  4.74686362e-02, -2.13208795e-02,
         3.40786614e-02,  1.61033608e-02, -1.28562227e-02,
        -3.41572911e-02,  5.08170202e-03,  3.42685096e-02,
        -4.27875519e-02,  3.15414332e-02, -4.68228236e-02,
        -3.08310986e-02,  3.12447436e-02,  3.53118442e-02,
         4.04568762e-03, -2.01974399e-02, -4.46451306e-02,
        -4.10275683e-02,  1.61081813e-02,  4.51924242e-02,
         1.22807994e-02,  2.67754234e-02,  4.98669781e-02,
         3.81411426e-02,  1.59688480e-02,  2.23631971e-02,
        -1.04747519e-01,  1.30989566e-01, -1.22102417e-01,
        -1.27681702e-01, -3.13720465e-01,  2.34213993e-01,
        -1.50504902e-01,  1.26815423e-01,  6.83802515e-02,
         2.24938430e-02, -9.28697884e-02,  2.29649872e-01,
         4.44575539e-03, -3.85670871e-01, -1.34370908e-01,
      

In [63]:
# Metrics & Task
metrics = tfrs.metrics.FactorizedTopK(candidates=book_infos.map(lambda features: book_model(features)))
task = tfrs.tasks.Retrieval(metrics=metrics)

### Book to Book Model

In [64]:
class Book2BookModel(tfrs.Model):
    def __init__(self, book_id_model, book_title_model, book_author_model, book_publisher_model, output_dimension=64):
        super().__init__()
        self.book_id_model = book_id_model
        self.book_title_model = book_title_model
        self.book_author_model = book_author_model
        self.book_publisher_model = book_publisher_model
        # combining book model with output dimension to fix output dimension
        self.book_model_raw = BookModel(self.book_id_model,
                                        self.book_title_model,
                                        self.book_author_model,
                                        self.book_publisher_model)
        self.book_model = tf.keras.Sequential([self.book_model_raw,
                                               tf.keras.layers.Dense(output_dimension)])
        # Metrics & Task
        self.candidates = book_infos.map(lambda x: self.book_model(x))
        metrics = tfrs.metrics.FactorizedTopK(candidates=self.candidates)
        # negative sampling also applied
        self.task = tfrs.tasks.Retrieval(metrics=metrics,
                                        num_hard_negatives=5)
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=True):
        # Generation of main book embedding from main item features
        main_book_embedding = self.book_model({'ISBN':features['main_ISBN'],
                                               'Book-Title':features['main_Book-Title'],
                                               'Book-Author': features['main_Book-Author'],
                                               'Publisher': features['main_Publisher']})

        # Generation of similar book embedding from similar item features
        similar_book_embedding = self.book_model({'ISBN':features['similar_ISBN'],
                                                  'Book-Title':features['similar_Book-Title'],
                                                  'Book-Author': features['similar_Book-Author'],
                                                  'Publisher': features['similar_Publisher']})

        # loss and the metric calculation
        # compute metrics set false to skyrock training speed
        return self.task(main_book_embedding,
                         similar_book_embedding,
                         compute_metrics=False)

In [65]:
# Book to Book Model initialization
book2book_model = Book2BookModel(book_id_model,
                                 book_title_model,
                                 book_author_model,
                                 book_publisher_model)
book2book_model.compile(optimizer=tf.keras.optimizers.legacy.Adagrad(learning_rate=0.1))

In [66]:
book2book_model.fit(train, epochs=5)

Epoch 1/5
284/284 [==============================] - 8s 19ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 4655.8043 - regularization_loss: 0.0000e+00 - total_loss: 4655.8043
Epoch 2/5
284/284 [==============================] - 6s 19ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 633.1789 - regularization_loss: 0.0000e+00 - total_loss: 633.1789
Epoch 3/5
284/284 [==============================] - 6s 19ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - fact

In [67]:
# Function to apply the model to each book feature set and return ISBN with embeddings
def extract_embeddings_with_isbn(features):
    embeddings = book2book_model.book_model(features)
    return features['ISBN'], embeddings

# Mapping the function over the dataset
book_embeddings = book_infos.map(extract_embeddings_with_isbn)
isbn_list = []
embeddings_list = []

# Example to inspect or use the embeddings with ISBNs
for isbn, embedding in book_embeddings:
    isbn_list.extend(list(isbn.numpy().astype(str)))  
    embeddings_list.extend(list(embedding.numpy()))

book_embedding_dataset = pd.DataFrame({'ISBN':isbn_list, 'embedding':embeddings_list})
book_embedding_dataset.head()

,ISBN,embedding
0,book_0002005018,"[0.24529923, 0.026332198, 0.10543761, 0.024581..."
1,book_0060973129,"[0.15041944, -0.20085151, 0.01697667, -0.02865..."
2,book_0374157065,"[0.22796887, 0.35494435, -0.09789037, 0.053342..."
3,book_0399135782,"[0.3008597, 0.14158057, 0.008636907, 0.1293548..."
4,book_1558746218,"[0.16193992, -0.110393316, -0.06777033, 0.0885..."


In [68]:
book_embedding_dict = dict(zip(book_embedding_dataset.ISBN, book_embedding_dataset.embedding))
book_title_dict = dict(zip(books['ISBN'], books['Book-Title']))

# ANN

In [69]:
dim = output_dimension

num_elements = book_embedding_dataset.shape[0]
# hnswlib initialization with cosine similarity
p = hnswlib.Index(space='cosine', dim=dim)

p.init_index(max_elements=num_elements, ef_construction=100, M=16)

p.set_ef(10)

embeddings = np.vstack(book_embedding_dataset["embedding"].values)
p.add_items(embeddings)

## Similar Book Search

In [70]:
def book_search(isbn, k=3):
    """Gets input embeddings and return top k similar items"""

    # Generate embedding for the user query
    query_embedding = book_embedding_dict[isbn]

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    labels, _ = p.knn_query(query_embedding, k=k+1)
    results = book_embedding_dataset.iloc[list(labels[0][1:])].to_dict('records')
    similar_isbns = [similar_isbn['ISBN'] for similar_isbn in results]
    return similar_isbns

In [71]:
# Visualization of Recommendation
# ISBN of "Harry Potter and the Sorcerer's Stone (Book 1)"
main_book = 'book_0590353403'
print(f"Main Book:\n{book_title_dict[main_book]}\n\nSimilar Books:")

similar_books = book_search(main_book, k=10)
for i, similar_book in enumerate(similar_books):
    print(f"{i+1}. {book_title_dict[similar_book]}")

Main Book:
Harry Potter and the Sorcerer's Stone (Book 1)

Similar Books:
1. Harry Potter and the Sorcerer's Stone (Book 1 Audio CD)
2. Harry Potter and the Prisoner of Azkaban (Book 3)
3. Harry Potter and the Order of the Phoenix (Book 5)
4. Sleeping Beauty: A Novel
5. The Lovely Bones: A Novel
6. Sacred
7. The great fire
8. BAG OF BONES : A NOVEL
9. Casual Rex: A Novel
10. Trouble Man the Life and Deat Gaye


## Recommendations & Evaluation

### Top Products

In [84]:
def get_popular_books(df_ratings, k=10):
  # Calculate the number of ratings for each movie
  rating_counts = df_ratings['ISBN'].value_counts().reset_index()
  rating_counts.columns = ['ISBN', 'rating_count']

  # Get the most frequently rated movies
  min_ratings_threshold = rating_counts['rating_count'].quantile(0.95)

  # Filter movies based on the minimum number of ratings
  popular_movies = ratings.merge(rating_counts, on='ISBN')
  popular_movies = popular_movies[popular_movies['rating_count'] >= min_ratings_threshold]

  # Calculate the average rating for each movie
  average_ratings = popular_movies.groupby('ISBN')['Book-Rating'].mean().reset_index()

  # Get the top 10 rated movies
  top_10_movies = list(average_ratings.sort_values('Book-Rating', ascending=False).head(k).ISBN.values)
  return top_10_movies

popular_books = get_popular_books(ratings)
print("Popular Books:")
for i, popular_book in enumerate(popular_books):
  print(f"{i+1}. {book_title_dict[popular_book]}")

Popular Books:
1. Anne of Avonlea (Anne of Green Gables Novels (Paperback))
2. El Codigo Da Vinci / The Da Vinci Code
3. Maus 1. Mein Vater kotzt Geschichte aus. Die Geschichte eines Ã?Â?berlebenden.
4. Complete Chronicles of Narnia
5. The Cat in the Hat
6. The Ultimate Hitchhiker's Guide
7. Dandelion Wine (Grand Master Editions)
8. The Grapes of Wrath
9. Der Alchimist.
10. The Little Prince (Wordsworth Collection)


### Similar Products 

In [73]:
# Function to apply the model to each book feature set and return ISBN with embeddings
def extract_pairs(features):
    return features['main_ISBN'], features['similar_ISBN']

main_isbn_list = []
similar_isbn_list = []

book_pairs_test = test.map(extract_pairs)
for main_isbn, similar_isbn in book_pairs_test:
    main_isbn_list.extend(list(main_isbn.numpy().astype(str)))  
    similar_isbn_list.extend(list(similar_isbn.numpy().astype(str)))

In [85]:
popular_reco_results = []
two_tower_reco_results = []
k = 50
popular_books = get_popular_books(ratings, k=k)

for main_isbn, similar_isbn in zip(main_isbn_list, similar_isbn_list):
    popular_reco_check = np.isin(popular_books, similar_isbn).astype(int)
    popular_reco_results.append(popular_reco_check)
    # get embedding based recommendations
    similar_books = book_search(main_isbn, k=k)
    two_tower_check = np.isin(similar_books, similar_isbn).astype(int)
    two_tower_reco_results.append(two_tower_check)

### NDCG Scores

In [88]:
from sklearn.metrics import ndcg_score

# Since we have already sorted our recommendations
# An array that represent our recommendation scores is used.
representative_array = [[i for i in range(k, 0, -1)]] * len(two_tower_reco_results)

for k in [10, 20, 50]:
  two_tower_result = ndcg_score(two_tower_reco_results,
                                  representative_array, k=k)
  popular_result = ndcg_score(popular_reco_results,
                              representative_array, k=k)
  
  print(f"Two Tower NDCG result at top {k}: {round(two_tower_result, 4)}")
  print(f"Popular recommendation NDCG result at top {k}: {round(popular_result, 4)}\n\n")

Two Tower NDCG result at top 10: 0.0112
Popular recommendation NDCG result at top 10: 0.0


Two Tower NDCG result at top 20: 0.012
Popular recommendation NDCG result at top 20: 0.0


Two Tower NDCG result at top 50: 0.0135
Popular recommendation NDCG result at top 50: 0.0014




## References
- https://www.tensorflow.org/recommenders/examples/basic_retrieval
- https://www.tensorflow.org/recommenders/examples/featurization